In [4]:
import requests
from PIL import Image
import gradio as gr
import io

# Prediction 설정
PREDICTION_KEY = "5k8oJDDDmqLn5Yy9n1Q16CHetW6H0pvTjFPj1Q4JpQl7dAVJE0WhJQQJ99BEACYeBjFXJ3w3AAAIACOGZmg4"
ENDPOINT_URL = "https://cv7934-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/92adf90f-3b67-4923-b2eb-1804da244279/detect/iterations/Iteration1/image"

headers = {
    "Prediction-Key": PREDICTION_KEY,
    "Content-Type": "application/octet-stream"
}

# 예측 함수
def predict_with_api(image: Image.Image):
    try:
        buf = io.BytesIO()
        image.save(buf, format='JPEG')
        byte_data = buf.getvalue()
        
        response = requests.post(ENDPOINT_URL, headers=headers, data=byte_data)

        if response.status_code != 200:
            return f"API 호출 실패: {response.status_code} - {response.text}"
        
        predictions = response.json().get("predictions", [])
        if not predictions:
            return "예측 결과 없음"

        top_prediction = max(predictions, key=lambda x: x["probability"])
        label = top_prediction["tagName"]
        probability = top_prediction["probability"]
        
        return f"{label} ({probability*100:.2f}%)"
    
    except Exception as e:
        return f"예외 발생: {str(e)}"

# Gradio 인터페이스 (웹캠 사용)
interface = gr.Interface(
    fn=predict_with_api,
    inputs=gr.Image(type="pil", sources=["webcam"]),  # 📷 카메라 입력
    outputs=gr.Text(),
    title="Custom Vision 실시간 예측",
    description="카메라로 사진을 찍어 예측 결과를 확인해보세요."
)

interface = gr.Interface(
    fn=predict_with_api,
    inputs=gr.Image(type="pil"),  # 카메라 입력 허용!
    outputs=gr.Text(),
    title="Custom Vision 실시간 예측",
    description="카메라로 사진을 찍어 예측 결과를 확인해보세요."
)
interface.launch(share=True)




* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://42131ed9d10dd2032d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
